# Nucleus Detection

### Next steps

* Work out a loss function
* Rework network to run on our image sizes
* Decide how to run on other image sizes
* Initialization of 

### Mid term plans

* Data augmentation

# Imports

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [7]:
import data_loaders, prepare_submission, net, mask_prep

import matplotlib.pyplot as plt
import numpy as np
import torch                     
from torch.autograd import Variable

# Data

In [3]:
train_raw, train_keys_raw = data_loaders.training_images()
use_saved_weights = True
if use_saved_weights:
    weights_raw = data_loaders.weights()
else: # slow slow slow...
    masks = data_loaders.training_masks()
    weights_raw = mask_prep.weights_from_all_masks(masks)
    print("You probably want to save these weights now")
# test, test_keys = data_loaders.test_images()

'\ntest, test_keys = data_loaders.test_images()\ns_masks = data_loaders.single_masks(masks)\n'

In [73]:
# For now, let us reduce our input data to just things that are 256x256
train, train_keys, weights = [], [], []
for i in range(len(train_raw)):
    if train_raw[i].shape == (256, 256):
        train.append(train_raw[i])
        train_keys.append(train_keys_raw[i])
        weights.append(weights_raw[i])
print(len(train), len(train_keys), len(weights))

334 334 334


# UNET

In [ ]:
n = net.u_net()

In [ ]:
# Run something through it as a sanity check
inp = Variable(torch.randn(1, 1, 572, 572))
n(inp).shape